In [1]:
import requests
import pandas as pd
import mysql.connector
import time
import ast
import warnings

In [2]:
database_name = 'league-database'
username = 'admin'
password = 'CowDog019283'
endpoint = 'league-database.cgdkdqtwgxcv.us-east-2.rds.amazonaws.com'
port = 3306
db_name = 'leaguedb'

In [3]:
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
one = pd.read_csv('one.csv')
two = pd.read_csv('two.csv')
three = pd.read_csv('three.csv')
four = pd.read_csv('four.csv')
five = pd.read_csv('five.csv')
six = pd.read_csv('six.csv')
seven = pd.read_csv('seven.csv')
rest = pd.read_csv('last.csv')

In [5]:
lit = ast.literal_eval(one['info'][15])

In [6]:
lit['info']['teams'][0]['objectives']

{'baron': {'first': False, 'kills': 0},
 'champion': {'first': False, 'kills': 25},
 'dragon': {'first': False, 'kills': 0},
 'inhibitor': {'first': False, 'kills': 0},
 'riftHerald': {'first': True, 'kills': 2},
 'tower': {'first': False, 'kills': 1}}

In [7]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()

In [9]:
query = '''CREATE TABLE OneRowGames(
            id VARCHAR(80),
            duration INT,
            gamemode VARCHAR(35),
            bluewin BOOLEAN,
            blueWardsPlaced INT,
            blueWardsDestroyed INT,
            blueFirstBlood BOOLEAN,
            blueKills INT,
            blueDeaths INT,
            blueAssists INT,
            blueDragons INT,
            blueHeralds INT,
            blueBarons INT,
            blueTowersDestroyed INT,
            blueInhibitorsDestroyed INT,
            blueTotalGold INT,
            blueAverageLevel FLOAT  ,
            blueTotalExp INT,
            blueMinions INT,
            blueDamage INT,
            blueTimeDead INT,
            blueCCDealt INT,
            
            redwin BOOLEAN,
            redWardsPlaced INT,
            redWardsDestroyed INT,
            redFirstBlood BOOLEAN,
            redKills INT,
            redDeaths INT,
            redAssists INT,
            redDragons INT,
            redHeralds INT,
            redBarons INT,
            redTowersDestroyed INT,
            redInhibitorsDestroyed INT,
            redTotalGold INT,
            redAverageLevel FLOAT,
            redTotalExp INT,
            redMinions INT,
            redDamage INT,
            redTimeDead INT,
            redCCDealt INT);
            '''
cursor.execute(query)

In [12]:
query = '''SHOW COLUMNS FROM OneRowGames'''
cursor.execute(query)
columns = cursor.fetchall()

In [15]:
names = []
for column in columns:
    names.append(column[0])

In [18]:
len(names)

41

In [8]:
# This is how we are going to take a single game and put it into one row
# This is a function that will do it for all games
def one_row_games(data, counter):
    try:
        if counter >= len(data):
            return None
        for info in data[counter:]:
            # Adding each row to AWS
            add_row = (
            '''INSERT INTO OneRowGames(
                id, duration, gamemode, bluewin, blueWardsPlaced, blueWardsDestroyed, blueFirstBlood, blueKills, blueDeaths, blueAssists, blueDragons, 
                blueHeralds, blueBarons, blueTowersDestroyed, blueInhibitorsDestroyed, blueTotalGold, blueAverageLevel, blueTotalExp, blueMinions, blueDamage, 
                blueTimeDead, blueCCDealt, redwin, redWardsPlaced, redWardsDestroyed, redFirstBlood, redKills, redDeaths, redAssists, redDragons, redHeralds, 
                redBarons, redTowersDestroyed, redInhibitorsDestroyed, redTotalGold, redAverageLevel, redTotalExp, redMinions, redDamage, redTimeDead, redCCDealt)
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
        )
            # Change string to literal
            lit = ast.literal_eval(info)
            # Unique Riot ID of the game
            id = lit['metadata']['matchId']
            # Duration of game
            duration = lit['info']['gameDuration']
            # Gamemode
            gamemode = lit['info']['gameMode']
            # Whoever won, their win will be True
            bluewin = lit['info']['teams'][0]['win']
            redwin = lit['info']['teams'][1]['win']
            # Amount of warding totems placed by each team
            blueWardsPlaced = 0
            redWardsPlaced = 0
            # Amount of wards destroyed by each team
            blueWardsDestroyed = 0
            redWardsDestroyed = 0
            # Which team got the first kill of the game
            blueFirstBlood = False
            redFirstBlood = False
            # Amount of kills per team
            blueKills = 0
            redKills = 0
            # Amount of deaths per team
            blueDeaths = 0
            redDeaths = 0
            # Amount of assists per team
            blueAssists = 0
            redAssists = 0
            # Amount of dragons, heralds, and barons killed per team
            blueDragons =  lit['info']['teams'][0]['objectives']['dragon']['kills']
            blueHeralds = lit['info']['teams'][0]['objectives']['riftHerald']['kills']
            redDragons = lit['info']['teams'][1]['objectives']['dragon']['kills']
            redHeralds = lit['info']['teams'][1]['objectives']['riftHerald']['kills']
            blueBarons = lit['info']['teams'][0]['objectives']['baron']['kills']
            redBarons = lit['info']['teams'][1]['objectives']['baron']['kills']
            # Towers and inhibitors destroyed by the blue team
            blueTowersDestoryed = lit['info']['teams'][0]['objectives']['tower']['kills']
            blueInhibitorsDestroyed = lit['info']['teams'][0]['objectives']['inhibitor']['kills']
            # Towers and inhibitors destroyed by the red team
            redTowersDestoryed = lit['info']['teams'][1]['objectives']['tower']['kills']
            redInhibitorsDestroyed = lit['info']['teams'][1]['objectives']['inhibitor']['kills']
            # Blue team total gold and red team total gold
            blueTotalGold = 0
            redTotalGold = 0
            # Blue and red team average levels and total experience
            blueAvgLevel = 0
            redAvgLevel = 0
            blueTotalExp = 0 
            redTotalExp = 0 
            # Total minions killed for each team
            blueMinions = 0
            redMinions = 0
            # Total Damage dealt to enemy team
            blueTotalDmg = 0
            redTotalDmg = 0
            # Total time spend dead in seconds
            blueTimeDead = 0
            redTimeDead = 0
            # Total time of Crowd Control dealt to enemy team
            blueCCDealt = 0
            redCCDealt = 0
            # For loop for blue team
            for row in lit['info']['participants'][:5]:
                blueWardsPlaced += row['wardsPlaced']
                blueWardsDestroyed += row['wardsKilled']
                blueKills += row['kills']
                blueDeaths += row['deaths']
                blueAssists += row['assists']
                blueTotalGold += row['goldEarned']
                blueAvgLevel += row['champLevel']
                blueTotalExp += row['champExperience']
                blueMinions += row['totalMinionsKilled']
                blueTotalDmg += row['totalDamageDealtToChampions']
                blueTimeDead += row['totalTimeSpentDead']
                blueCCDealt += row['timeCCingOthers']
                if row['firstBloodKill'] == True:
                    blueFirstBlood = True
            # For loop for red team
            for row in lit['info']['participants'][5:]:
                redWardsPlaced += row['wardsPlaced']
                redWardsDestroyed += row['wardsKilled']
                redKills += row['kills']
                redDeaths += row['deaths']
                redAssists += row['assists']
                redTotalGold += row['goldEarned']
                redAvgLevel += row['champLevel']
                redTotalExp += row['champExperience']
                redMinions += row['totalMinionsKilled']
                redTotalDmg += row['totalDamageDealtToChampions']
                redTimeDead += row['totalTimeSpentDead']
                redCCDealt += row['timeCCingOthers']
                if row['firstBloodKill'] == True:
                    redFirstBlood = True

            blueAvgLevel = blueAvgLevel / 5
            redAvgLevel = redAvgLevel / 5
            counter += 1
            row = (id, duration, gamemode, bluewin, blueWardsPlaced, blueWardsDestroyed, blueFirstBlood, blueKills, blueDeaths,
            blueAssists, blueDragons, blueHeralds, blueBarons, blueTowersDestoryed, blueInhibitorsDestroyed, blueTotalGold,
            blueAvgLevel, blueTotalExp, blueMinions, blueTotalDmg, blueTimeDead, blueCCDealt, redwin, redWardsPlaced, redWardsDestroyed,
            redFirstBlood, redKills, redDeaths, redAssists, redDragons, redHeralds, redBarons, redTowersDestoryed, redInhibitorsDestroyed,
            redTotalGold, redAvgLevel, redTotalExp, redMinions, redTotalDmg, redTimeDead, redCCDealt)

            cursor.execute(add_row, row)
            connection.commit()
            if counter == len(data):
                return None

    except IndexError:
        counter += 1
        print('index problem, keep going', counter)
        if counter < len(data):
            one_row_games(data, counter)

        else:
            return None
    else: 
        raise


In [9]:
files = [one, two, three, four, five, six, seven, rest]
count = 0
for file in files:
    count += 1
    print('file number', count)
    one_row_games(file['info'], 0)

file number 1
index problem, keep going 738
index problem, keep going 1697
index problem, keep going 1763
index problem, keep going 2993
index problem, keep going 2994
index problem, keep going 2995
index problem, keep going 3065
index problem, keep going 3460
index problem, keep going 3851
index problem, keep going 3908
index problem, keep going 5705
index problem, keep going 7353
index problem, keep going 7354
index problem, keep going 7355
index problem, keep going 7356
index problem, keep going 7357
index problem, keep going 7359
index problem, keep going 7479
index problem, keep going 9058
index problem, keep going 9490
index problem, keep going 9578
index problem, keep going 9935
file number 2
index problem, keep going 1285
index problem, keep going 4056
index problem, keep going 4057
index problem, keep going 4060
index problem, keep going 7197
index problem, keep going 8885
index problem, keep going 9600
file number 3
index problem, keep going 674
index problem, keep going 676


In [4]:
connection = mysql.connector.connect(host=endpoint, database=db_name, user=username, password=password)
cursor = connection.cursor()
query = "SELECT * FROM OneRowGames"
pd.read_sql(query, connection)

,id,duration,gamemode,bluewin,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,...,redBarons,redTowersDestroyed,redInhibitorsDestroyed,redTotalGold,redAverageLevel,redTotalExp,redMinions,redDamage,redTimeDead,redCCDealt
0,NA1_4366795660,1235,ARAM,0,0,0,0,42,46,108,...,0,4,2,70898,17.4,96267,210,146448,1066,185
1,NA1_4366527467,1435,ARAM,0,0,0,0,63,68,128,...,0,4,2,88550,18.0,131673,321,158948,1820,70
2,NA1_4366533737,1206,ARAM,1,0,0,1,51,40,103,...,0,0,0,65596,17.2,89506,215,103026,1178,123
3,NA1_4365266709,1433,ARAM,0,0,0,1,39,51,69,...,0,4,1,80239,18.0,118917,254,140354,1010,180
4,NA1_4365293282,910,ARAM,0,0,0,1,20,43,47,...,0,4,1,56191,15.0,70672,170,99506,442,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76332,NA1_4363734624,193,CLASSIC,0,2,1,0,0,0,0,...,0,0,0,4979,2.6,3890,50,2782,0,6
76333,NA1_4363741186,1468,CLASSIC,0,66,5,0,17,35,29,...,1,10,1,57200,14.6,66668,583,92061,568,112
76334,NA1_4363687957,641,CLASSIC,1,18,5,0,5,1,7,...,0,0,0,14815,6.6,17268,204,13485,81,10
76335,NA1_4363672945,1984,CLASSIC,0,100,46,1,29,31,48,...,2,9,1,67549,15.4,74515,753,113258,889,95
